# api 크롤링

기존의 크롤링 방식은 

1. 내가 직접 홈페이지에 브라우저를 켜서 접근하거나(selenium) 

혹은 

2. 파이썬 자체적으로 호출을 요청해서(requests)

데이터를 가지고 오는 방식이었습니다.

이 방식은 서버에 부하가 많이 갈 뿐더러 나에게 필요없는 데이터까지 한 번에 호출해서 가져오는 문제가 있습니다.

따라서 서비스 제공자측에서는 서버 부하를 줄이고, 사용자에게 맞춤형을 데이터를 제공하기 위해 api서버를 운영합니다.

api서버는 인가된 데이터만을 개발자에게 넘겨서 서버도 안정적으로 유지하며(횟수 제한이 있는 사이트도 많습니다. 라이엇 데이터는 1초에 5회, 2분에 100회로 제한됩니다.) 

필요없는 동영상자료나 그림자료를 호출하지 않으므로 트래픽을 줄일 수 있습니다.



api 서버 접근시 보통 urllib.request 를 이용하게 됩니다.

In [1]:
# 사이트에 자료 요청
import urllib.request 
# json 데이터 핸들링
import json 
# DataFrame 자료형 활용
import pandas as pd 
# json 데이터를 pandas DataFrame으로 변환
#from pandas.io.json import json_normalize 

# 영진위 api신청

http://www.kobis.or.kr/kobisopenapi/homepg/main/main.do

1. 접속 후 가입 및 로그인

2. 키발급 탭에서 키 발급하기

3. 요청 api종류 보고 요청양식 및 데이터 확인하기

In [2]:
api_key = "f5eef3421c602c6cb7ea224104795888" # 발급받은 키만 복붙하세요
request_date = "20220622" # 조회날짜를  xxxxyydd 20220623 형식으로 넣어주세요
url = "http://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=%s&targetDt=%s" % (api_key, request_date)
print(url)

http://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=f5eef3421c602c6cb7ea224104795888&targetDt=20220622


# json데이터를 팬더스 데이터프레임으로 변환

api 데이터는 보통 json(JavaScript Object Notation) 타입으로 받아집니다.

쉽게 말하면 자바스크립트 데이터를 전달하기 좋게 설정한 자료형인데

파이썬의 딕셔너리와 거의 같다고 보시면 됩니다.

따라서 json 데이터를 팬더스 데이터로 변환한다는 것은 사실상

딕셔너리 데이터를 팬더스 데이터프레임으로 변환하는것입니다.

In [3]:
# 위의 url 변수를 이용해 데이터를 요청합니다.
r = urllib.request.urlopen(url)

# 요청 페이지의 결과 데이터를 파이썬 내부 데이터로 바꿉니다.
json_raw = r.read()

In [4]:
# b'~~~~~~~'로 시작하는 이 데이터는 파이썬에서 byte자료형으로 처리되고
# 데이터프레임으로는 바로 변환이 불가능합니다.
json_raw

b'{"boxOfficeResult":{"boxofficeType":"\xec\x9d\xbc\xeb\xb3\x84 \xeb\xb0\x95\xec\x8a\xa4\xec\x98\xa4\xed\x94\xbc\xec\x8a\xa4","showRange":"20220622~20220622","dailyBoxOfficeList":[{"rnum":"1","rank":"1","rankInten":"0","rankOldAndNew":"NEW","movieCd":"20194376","movieNm":"\xed\x83\x91\xea\xb1\xb4: \xeb\xa7\xa4\xeb\xb2\x84\xeb\xa6\xad","openDt":"2022-06-22","salesAmt":"1976049530","salesShare":"54.3","salesInten":"1976049530","salesChange":"100","salesAcc":"2716938360","audiCnt":"188312","audiInten":"188312","audiChange":"100","audiAcc":"253016","scrnCnt":"1975","showCnt":"9158"},{"rnum":"2","rank":"2","rankInten":"-1","rankOldAndNew":"OLD","movieCd":"20224882","movieNm":"\xeb\xa7\x88\xeb\x85\x80(\xe9\xad\x94\xe5\xa5\xb3) Part2. The Other One","openDt":"2022-06-15","salesAmt":"836476290","salesShare":"23.0","salesInten":"-253649090","salesChange":"-23.3","salesAcc":"18226409650","audiCnt":"82691","audiInten":"-25491","audiChange":"-23.6","audiAcc":"1769461","scrnCnt":"976","showCnt":"35

In [5]:
# 바이트자료형임을 확인
type(json_raw)

bytes

In [6]:
# 따라서 utf-8 형식으로 고쳐줘야 합니다. .decode()를 이용합니다.
# encode => 우리가 쓰는 문자를 컴퓨터가 쓰는 형태로 변환
# decode => 컴퓨터가 쓰는 형태의 문자를 우리가 쓰는 형태로 변환
json_utf8 = json_raw.decode("utf-8")

In [7]:
# 딕셔너리 구조를 유지하고 있는 문자로 변환됨
type(json_utf8)

str

In [8]:
json_utf8

'{"boxOfficeResult":{"boxofficeType":"일별 박스오피스","showRange":"20220622~20220622","dailyBoxOfficeList":[{"rnum":"1","rank":"1","rankInten":"0","rankOldAndNew":"NEW","movieCd":"20194376","movieNm":"탑건: 매버릭","openDt":"2022-06-22","salesAmt":"1976049530","salesShare":"54.3","salesInten":"1976049530","salesChange":"100","salesAcc":"2716938360","audiCnt":"188312","audiInten":"188312","audiChange":"100","audiAcc":"253016","scrnCnt":"1975","showCnt":"9158"},{"rnum":"2","rank":"2","rankInten":"-1","rankOldAndNew":"OLD","movieCd":"20224882","movieNm":"마녀(魔女) Part2. The Other One","openDt":"2022-06-15","salesAmt":"836476290","salesShare":"23.0","salesInten":"-253649090","salesChange":"-23.3","salesAcc":"18226409650","audiCnt":"82691","audiInten":"-25491","audiChange":"-23.6","audiAcc":"1769461","scrnCnt":"976","showCnt":"3502"},{"rnum":"3","rank":"3","rankInten":"-1","rankOldAndNew":"OLD","movieCd":"20204548","movieNm":"범죄도시 2","openDt":"2022-05-18","salesAmt":"577713370","salesShare":"15.9","sale

In [9]:
# json.loads(딕셔너리 형태의 문자열 자료)
# 딕셔너리 형태로 적힌 문자열 자료를 딕셔너리형으로 변환
json_complete = json.loads(json_utf8)

In [10]:
# 딕셔너리(json) 변환 완료 확인
type(json_complete)

dict

In [11]:
# json_normalize(딕셔너리 자료)
# 딕셔너리 자료를 데이터프레임 형태로 변환해줌.
pd.json_normalize(json_complete)

,boxOfficeResult.boxofficeType,boxOfficeResult.showRange,boxOfficeResult.dailyBoxOfficeList
0,일별 박스오피스,20220622~20220622,"[{'rnum': '1', 'rank': '1', 'rankInten': '0', ..."


In [12]:
# boxOfficeResult.dailyBoxOfficeList 컬럼 내부에 딕셔너리가 여럿 묶인 리스트가 보이므로
# boxOfficeResult.dailyBoxOfficeList 컬럼을 타겟으로 데이터프레임화 합니다.
box_result = pd.json_normalize(json_complete['boxOfficeResult']['dailyBoxOfficeList'])

In [13]:
# json_normalize()는 pd.DataFrame() 으로 대체 가능합니다.
pd.DataFrame(json_complete)

,boxOfficeResult
boxofficeType,일별 박스오피스
dailyBoxOfficeList,"[{'rnum': '1', 'rank': '1', 'rankInten': '0', ..."
showRange,20220622~20220622


API 크롤링으로 서비스 제공시

사용자가 조회할때마다 매번 API서버에 요청을 넣어서 데이터를 가져오면

호출횟수를 금방 소모하게 됩니다.

따라서, 보통 API를 활용한 서비스는 미리 데이터를 DB에 적재해놓고

사용자가 요청시 내 DB에 있는 데이터를 우선적으로 보여주고, 없으면 외부서버에 

요청해서 내 DB에 INSERT한 다음 다시 내 DB데이터를 사용자에게 보여줍니다.

In [14]:
url

'http://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=f5eef3421c602c6cb7ea224104795888&targetDt=20220622'

날짜 데이터는 28일까지인 달(2월), 30일까지인 달, 31일까지인 달 들이 있어서

단순히 문자열에 1씩 더하거나 날짜처리를 하기도 어렵고

위의 처리를 다 했다고 해도 4년에 1번 윤달이 있습니다(2월 29일)

그래서, 날짜는 전적으로 날짜데이터를 이용해 처리해야만 합니다.

파이썬에서는 datetime을 이용해 처리합니다.(자바는 java.util.Date를 써서 처리)

In [15]:
import time
import datetime # 파이썬 날짜 자료형

In [16]:
start_datetime = datetime.datetime(2021, 8, 9) # 년 월 일 순으로 넣어줍니다.
# 형식 바꾸기
target_date = start_datetime.strftime('%Y%m%d') #영진위 형식으로 변경
print(target_date)
# 날짜 하루 올리기
next_datetime = start_datetime - datetime.timedelta(days=1)
next_datetime

20210809


datetime.datetime(2021, 8, 8, 0, 0)

In [17]:
target_dates = ['20220805','20220806','20220807']
# 날짜는 시작날짜와 + 며칠 을 처리할지 결정할 수 있습니다.
# 3일치 데이터를 얻은 다음 to_csv나 to_excel로 저장해주세요.

# 반복문과 append를 이용해서 3일치 데이터를 요청해서 합쳐보세요.
# target_date 컬럼에 조회 날짜를 추가해주세요.
# 빈 데이터프레임을 하나 생성해놓고(pd.DataFrame())
merged_data = pd.DataFrame()
for date in target_dates:
    api_key = "f5eef3421c602c6cb7ea224104795888" # 발급받은 키만 복붙하세요
    request_date = date # 조회날짜를  xxxxyydd 20220623 형식으로 넣어주세요
    url = "http://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=%s&targetDt=%s" % (api_key, request_date)
    # 위의 url 변수를 이용해 데이터를 요청합니다.
    r = urllib.request.urlopen(url)

    # 요청 페이지의 결과 데이터를 파이썬 내부 데이터로 바꿉니다.
    json_raw = r.read()
    
    json_utf8 = json_raw.decode("utf-8")
    # json.loads(딕셔너리 형태의 문자열 자료)
    # 딕셔너리 형태로 적힌 문자열 자료를 딕셔너리형으로 변환
    json_complete = json.loads(json_utf8)
    box_result = pd.json_normalize(json_complete['boxOfficeResult']['dailyBoxOfficeList'])
    # box_result에 날짜 컬럼 추가해 날짜 정보 넣기
    box_result['date'] = request_date
    merged_data = pd.concat([merged_data, box_result])
merged_data

,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt,date
0,1,1,1,OLD,20209343,한산: 용의 출현,2022-07-27,2565550184,41.6,530985656,26.1,37567514644,250627,41280,19.7,3692258,1530,6343,20220805
1,2,2,-1,OLD,20196410,비상선언,2022-08-03,2094794151,34.0,71338791,3.5,7693532482,213574,-8542,-3.8,794758,1718,7281,20220805
2,3,3,1,OLD,20194376,탑건: 매버릭,2022-06-22,504483247,8.2,148299938,41.6,77905758474,46942,12422,36,7305825,605,1238,20220805
3,4,4,-1,OLD,20205362,미니언즈2,2022-07-20,413915957,6.7,948579,0.2,17793954400,43092,-1769,-3.9,1837970,783,1595,20220805
4,5,5,0,OLD,20225190,뽀로로 극장판 드래곤캐슬 대모험,2022-07-28,167553618,2.7,-5960802,-3.4,2909201168,18088,-1181,-6.1,312147,537,779,20220805
5,6,6,0,OLD,20209654,헤어질 결심,2022-06-29,142017614,2.3,34703455,32.3,18014179742,13226,3017,29.6,1741846,265,432,20220805
6,7,7,0,OLD,20208446,외계+인 1부,2022-07-20,113278760,1.8,22370761,24.6,15283607796,11557,2267,24.4,1466378,407,576,20220805
7,8,8,0,OLD,20226107,극장판 도라에몽: 진구의 우주소전쟁 리틀스타워즈 2021,2022-08-03,49992320,0.8,-4086959,-7.6,192969299,5266,-680,-11.4,20859,362,513,20220805
8,9,9,0,OLD,20225237,명탐정 코난: 할로윈의 신부,2022-07-13,27421500,0.4,3966180,16.9,4509942774,2517,232,10.2,434339,75,94,20220805
9,10,10,26,OLD,20224882,마녀(魔女) Part2. The Other One,2022-06-15,12398000,0.2,11939500,2604,28919650670,2433,2378,4323.6,2806159,20,20,20220805


In [158]:
# 날짜는 시작날짜와 + 며칠 을 처리할지 결정할 수 있습니다.
# 1년치 데이터를 얻은 다음 to_csv나 to_excel로 저장해주세요.
start_datetime = datetime.datetime(2021, 8, 8) # 년 월 일 순으로 넣어줍니다.

# 반복문과 append를 이용해서 1년치 데이터를 요청해서 합쳐보세요.
# target_date 컬럼에 조회 날짜를 추가해주세요.
# 빈 데이터프레임을 하나 생성해놓고(pd.DataFrame())
merged_data = pd.DataFrame()
for idx in range(365):
    # 날짜를 문자로 변경
    target_date = start_datetime.strftime('%Y%m%d')
    
    api_key = "f5eef3421c602c6cb7ea224104795888" # 발급받은 키만 복붙하세요
    request_date = target_date # 조회날짜를  xxxxyydd 20220623 형식으로 넣어주세요
    url = "http://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=%s&targetDt=%s" % (api_key, request_date)
    # 위의 url 변수를 이용해 데이터를 요청합니다.
    r = urllib.request.urlopen(url)

    # 요청 페이지의 결과 데이터를 파이썬 내부 데이터로 바꿉니다.
    json_raw = r.read()
    
    json_utf8 = json_raw.decode("utf-8")
    # json.loads(딕셔너리 형태의 문자열 자료)
    # 딕셔너리 형태로 적힌 문자열 자료를 딕셔너리형으로 변환
    json_complete = json.loads(json_utf8)
    box_result = pd.json_normalize(json_complete['boxOfficeResult']['dailyBoxOfficeList'])
    # box_result에 날짜 컬럼 추가해 날짜 정보 넣기
    box_result['date'] = request_date
    merged_data = pd.concat([merged_data, box_result])
    # 해당 날짜 데이터를 가져와 통합한 뒤에는 날짜를 하루 더해주기
    start_datetime = start_datetime + datetime.timedelta(days=1)
    print("%s일차 자료 수집 완료" % idx)
    # 너무 자주 크롤링시 블록먹을 가능성이 있음.
    time.sleep(0.3)
    
merged_data

0일차 자료 수집 완료
1일차 자료 수집 완료
2일차 자료 수집 완료
3일차 자료 수집 완료
4일차 자료 수집 완료
5일차 자료 수집 완료
6일차 자료 수집 완료
7일차 자료 수집 완료
8일차 자료 수집 완료
9일차 자료 수집 완료
10일차 자료 수집 완료
11일차 자료 수집 완료
12일차 자료 수집 완료
13일차 자료 수집 완료
14일차 자료 수집 완료
15일차 자료 수집 완료
16일차 자료 수집 완료
17일차 자료 수집 완료
18일차 자료 수집 완료
19일차 자료 수집 완료
20일차 자료 수집 완료
21일차 자료 수집 완료
22일차 자료 수집 완료
23일차 자료 수집 완료
24일차 자료 수집 완료
25일차 자료 수집 완료
26일차 자료 수집 완료
27일차 자료 수집 완료
28일차 자료 수집 완료
29일차 자료 수집 완료
30일차 자료 수집 완료
31일차 자료 수집 완료
32일차 자료 수집 완료
33일차 자료 수집 완료
34일차 자료 수집 완료
35일차 자료 수집 완료
36일차 자료 수집 완료
37일차 자료 수집 완료
38일차 자료 수집 완료
39일차 자료 수집 완료
40일차 자료 수집 완료
41일차 자료 수집 완료
42일차 자료 수집 완료
43일차 자료 수집 완료
44일차 자료 수집 완료
45일차 자료 수집 완료
46일차 자료 수집 완료
47일차 자료 수집 완료
48일차 자료 수집 완료
49일차 자료 수집 완료
50일차 자료 수집 완료
51일차 자료 수집 완료
52일차 자료 수집 완료
53일차 자료 수집 완료
54일차 자료 수집 완료
55일차 자료 수집 완료
56일차 자료 수집 완료
57일차 자료 수집 완료
58일차 자료 수집 완료
59일차 자료 수집 완료
60일차 자료 수집 완료
61일차 자료 수집 완료
62일차 자료 수집 완료
63일차 자료 수집 완료
64일차 자료 수집 완료
65일차 자료 수집 완료
66일차 자료 수집 완료
67일차 자료 수집 완료
68일차 자료 수집 완료
69일차 자료 수집 완료
70일차 자료 수집 완료
71일차 자료 수집 완료
72

,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt,date
0,1,1,0,OLD,20204117,모가디슈,2021-07-28,2000829380,56.5,38859180,2,16579074530,196672,4041,2.1,1712375,1639,6757,20210808
1,2,2,0,OLD,20217845,더 수어사이드 스쿼드,2021-08-04,676797240,19.1,-127499190,-15.9,2942253950,63095,-12197,-16.2,281377,1033,3306,20210808
2,3,3,0,OLD,20218391,보스 베이비 2,2021-07-21,284206020,8.0,1506480,0.5,7511398580,30046,108,0.4,836805,777,1654,20210808
3,4,4,0,OLD,20191951,블랙 위도우,2021-07-07,194489780,5.5,-28197330,-12.7,29457331370,18740,-2673,-12.5,2908679,540,924,20210808
4,5,5,0,OLD,20218875,극장판 도라에몽: 진구의 신공룡,2021-08-05,88917620,2.5,-4415810,-4.7,337195080,9837,-612,-5.9,38669,502,781,20210808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,6,6,1,OLD,20209654,헤어질 결심,2022-06-29,169174033,1.7,-32677140,-16.2,18385204948,15203,-2798,-15.5,1775050,266,426,20220807
6,7,7,-1,OLD,20208446,외계+인 1부,2022-07-20,123965118,1.3,-98324419,-44.2,15629862451,11690,-7177,-38,1496935,374,499,20220807
7,8,8,0,OLD,20226107,극장판 도라에몽: 진구의 우주소전쟁 리틀스타워즈 2021,2022-08-03,77727199,0.8,2170399,2.9,346253298,7903,248,3.2,36417,357,466,20220807
8,9,9,0,OLD,20225237,명탐정 코난: 할로윈의 신부,2022-07-13,44635198,0.5,623298,1.4,4598589872,4122,13,0.3,442570,91,115,20220807


In [159]:
# 스크린당 평균 관객이 제일 많았던 영화를 뽑아주세요.
# 관객 총합 / 스크린 총합 = 스크린당 관객이 나옵니다.
# 컬럼 타입이 object이므로, 집계함수를 쓸 수 있도록 int로 컬럼 전체 자료형을 바꿔줘야 합니다.
# 현재 수치데이터도 전부 문자로 간주되고 있기 때문에 int로 자료형을 바꿔서 계산해야함
merged_data['audiCnt'] = merged_data['audiCnt'].astype(int)
merged_data['scrnCnt'] = merged_data['scrnCnt'].astype(int)

In [160]:
audi_scrn_sum = merged_data.pivot_table(index='movieNm', 
                                        values=['audiCnt','scrnCnt'], 
                                        aggfunc='sum').sort_values(by='audiCnt', 
                                                                   ascending=False)

In [161]:
# 관객 총합 / 스크린 총합 = 스크린당 관객이 나옵니다
# 컬럼명은 audiPerScrn
audi_scrn_sum['audiPerScrn'] = audi_scrn_sum['audiCnt'] / audi_scrn_sum['scrnCnt']
audi_scrn_sum

,audiCnt,scrnCnt,audiPerScrn
movieNm,,,
범죄도시 2,12686014,81645,155.380170
스파이더맨: 노 웨이 홈,7541536,90668,83.177483
탑건: 매버릭,7448595,57863,128.728116
닥터 스트레인지: 대혼돈의 멀티버스,5880700,48993,120.031433
한산: 용의 출현,4597833,22656,202.941075
...,...,...,...
지지,334,1,334.000000
포 언투 어스,300,1,300.000000
맛있는 여동생,259,20,12.950000


In [162]:
# 스크린당 관객수로 정렬해 가장 효율이 좋았던 영화들을 뽑아주세요.
# 집계대상의 스크린 개수는 최소 100개로 하겠습니다.
audi_scrn_sum[audi_scrn_sum['scrnCnt']>= 100].sort_values(by='audiPerScrn', ascending=False).head(20)

,audiCnt,scrnCnt,audiPerScrn
movieNm,,,
한산: 용의 출현,4597833,22656,202.941075
비상선언,1397964,8489,164.679468
범죄도시 2,12686014,81645,155.380170
탑건: 매버릭,7448595,57863,128.728116
닥터 스트레인지: 대혼돈의 멀티버스,5880700,48993,120.031433
쥬라기 월드: 도미니언,2832531,24893,113.788254
마녀(魔女) Part2. The Other One,2795796,27322,102.327648
토르: 러브 앤 썬더,2711863,26516,102.272703
미니언즈2,1971484,19463,101.293942


In [163]:
# 스크린 개수가 50개 이상인 영화중 스크린당 관객수가 평균이상인 영화만 나열해주세요
audi_scrn_sum[audi_scrn_sum['scrnCnt'] >= 50]['audiPerScrn'].mean()

26.827940403912617

In [164]:
final_data = audi_scrn_sum[(audi_scrn_sum['scrnCnt'] >= 50) & (audi_scrn_sum['audiPerScrn']>=26.8)].head(50)

In [25]:
# csv로 export하기
final_data.to_csv('평균이상영화.csv', encoding="utf-8")

In [26]:
# excel로 export하기
final_data.to_excel('평균이상영화.xlsx', encoding="utf-8")

# 가져온 데이터 mysql에 연동하기

- import pymysql
- 을 활용해서 연동할 수 있습니다.

- 임포트가 되지 않는다면 다시 설치를 해야합니다.
1. Anaconda Navigator에서 좌측 탭의 enviroments를 누릅니다.
2. Open terminal을 이용해 cmd창같은 창을 켭니다.(터미널창이라고 부릅니다.)
3. pip install pymysql 을 입력하고 실행합니다.

### 오라클 연동

위와 같은데 3번에서
- pip install cx_Oracle 
- 을 대신 입력해서 설치하면 됩니다.


In [26]:
# pymysql 실행을 해야 DB와 연결할 수 있습니다.
import pymysql

In [27]:
# MySQL 연결에 필요한 정보를 입력합니다.
con = pymysql.connect(host='localhost', user='root', password='mysql', db='pyprac2', charset='utf8',
                     cursorclass=pymysql.cursors.DictCursor) # 컬럼명까지 받아올지 여부

In [28]:
# SQL구문을 받아주는 cursor 객체를 사용합니다.
cur = con.cursor()

In [29]:
# SQL구문은 문자로 cur.execute()를 실행해서 수행시킵니다.
cur.execute("SELECT * FROM test_tbl2")

9

In [30]:
# execute가 실행되면, cur가 데이터를 저장하고 있는데 이걸 파이선 형식으로 뺴야합니다.
mysql_data = cur.fetchall()

In [31]:
mysql_data

[{'tno': 1, 'tname': 'a'},
 {'tno': 2, 'tname': 'b'},
 {'tno': 3, 'tname': 'c'},
 {'tno': 4, 'tname': 'd'},
 {'tno': 5, 'tname': 'e'},
 {'tno': 6, 'tname': 'f'},
 {'tno': 7, 'tname': 'g'},
 {'tno': 8, 'tname': 'g'},
 {'tno': 9, 'tname': 'g'}]

In [32]:
# 데이터프레임 형태로 변환
pd.DataFrame(mysql_data)

,tno,tname
0,1,a
1,2,b
2,3,c
3,4,d
4,5,e
5,6,f
6,7,g
7,8,g
8,9,g


In [33]:
# 여러분들이 data 'g'를 넣어보세요.
cur.execute("INSERT INTO test_tbl2 VALUES (null, 'g')")

1

In [34]:
# 커서가 아닌 커넥션에 commit을 주는것에 주의해주세요.
con.commit()

In [35]:
# 다시 select구문을 날려서 DB에 추가되었는지 확인까지해주세요
cur.execute("SELECT * FROM test_tbl2")

10

In [36]:
result_data = cur.fetchall()

In [37]:
pd.DataFrame(result_data)

,tno,tname
0,1,a
1,2,b
2,3,c
3,4,d
4,5,e
5,6,f
6,7,g
7,8,g
8,9,g
9,10,g


In [165]:
merged_data.head()

,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt,date
0,1,1,0,OLD,20204117,모가디슈,2021-07-28,2000829380,56.5,38859180,2,16579074530,196672,4041,2.1,1712375,1639,6757,20210808
1,2,2,0,OLD,20217845,더 수어사이드 스쿼드,2021-08-04,676797240,19.1,-127499190,-15.9,2942253950,63095,-12197,-16.2,281377,1033,3306,20210808
2,3,3,0,OLD,20218391,보스 베이비 2,2021-07-21,284206020,8.0,1506480,0.5,7511398580,30046,108,0.4,836805,777,1654,20210808
3,4,4,0,OLD,20191951,블랙 위도우,2021-07-07,194489780,5.5,-28197330,-12.7,29457331370,18740,-2673,-12.5,2908679,540,924,20210808
4,5,5,0,OLD,20218875,극장판 도라에몽: 진구의 신공룡,2021-08-05,88917620,2.5,-4415810,-4.7,337195080,9837,-612,-5.9,38669,502,781,20210808


In [73]:
# movie_tbl 데이터를 다 삭제하는 쿼리문을 날려주세요
cur.execute("DELETE FROM movie_tbl")
con.commit()

OperationalError: (1205, 'Lock wait timeout exceeded; try restarting transaction')

In [39]:
# 로우개수
len(merged_data) # 3650번 반복해야 DB에 insert가 된다는걸 알 수 있음.
# iloc[], .loc[] 를 이ㅛㅇ하면 특정 로우자료를 조회할 수 있고, 거기서 ['컬럼명']을 집어넣으면
# 특정 로우의 특정 컬럼값을 조회할 수 있습니다.
merged_data.iloc[4]['movieNm']

'극장판 도라에몽: 진구의 신공룡'

In [40]:
# 과제. MySQL이나 Oracle SQL에 현재 보고있는 db_insert데이터를
# 테이블 생성 및 반복문을 이용해서 다 DB에 넣어주시면 됩니다.
# 반복문을 이용해서 merged_data의 모든 데이터를 MySQL에 집어넣어주세요.
# 집어넣을 컬럼은 rnum, movieNm, openDt, audiCnt, showCnt, date

In [42]:
cur.execute("INSERT INTO movie_tbl (movieNm, openDt, audiCnt, showCnt, dataDate) VALUES ('%s', '%s', %s, %s, '%s')" % ("테스트", '2022-08-10', 300, 1, '20220810'))

1

In [43]:
con.commit()

In [41]:
# merged_data변수 내부의 영화 제목만 한 줄 한 줄 콘솔에 찍어보세요
# len() 을 이용해 데이터프레임의 로우 개수를 구할 수 있습니다.
range(len(merged_data))
# 위 range를 이용해 iloc[번호] 로 로우를 순서대로 조회가능합니다.
for i in range(len(merged_data)):
    selected_data = merged_data.iloc[i]
    print(selected_data['movieNm'])
    #나머지(개봉일, 관객수, 상영수, 데이터날짜)
    print(selected_data['openDt'])
    print(selected_data['audiCnt'])
    print(selected_data['showCnt'])
    print(selected_data['date'])
    print("-------------------------")

모가디슈
2021-07-28
196672
6757
20210808
-------------------------
더 수어사이드 스쿼드
2021-08-04
63095
3306
20210808
-------------------------
보스 베이비 2
2021-07-21
30046
1654
20210808
-------------------------
블랙 위도우
2021-07-07
18740
924
20210808
-------------------------
극장판 도라에몽: 진구의 신공룡
2021-08-05
9837
781
20210808
-------------------------
정글 크루즈
2021-07-28
9825
704
20210808
-------------------------
방법: 재차의
2021-07-28
5122
427
20210808
-------------------------
블랙핑크 더 무비
2021-08-04
3081
194
20210808
-------------------------
그린 나이트
2021-08-05
1855
234
20210808
-------------------------
랑종
2021-07-14
1508
105
20210808
-------------------------
모가디슈
2021-07-28
75348
6247
20210809
-------------------------
더 수어사이드 스쿼드
2021-08-04
24477
3007
20210809
-------------------------
보스 베이비 2
2021-07-21
9547
1315
20210809
-------------------------
블랙 위도우
2021-07-07
7568
873
20210809
-------------------------
정글 크루즈
2021-07-28
3150
569
20210809
-------------------------
싱크홀
2021-08-11
3092
31
20210809
----

2021-08-25
1402
160
20210903
-------------------------
여름날 우리
2021-08-25
1366
119
20210903
-------------------------
상견니
2021-08-31
639
114
20210903
-------------------------
최선의 삶
2021-09-01
637
138
20210903
-------------------------
샹치와 텐 링즈의 전설
2021-09-01
233073
7080
20210904
-------------------------
인질
2021-08-18
56543
2719
20210904
-------------------------
모가디슈
2021-07-28
42573
1983
20210904
-------------------------
싱크홀
2021-08-11
27576
1578
20210904
-------------------------
맨 인 더 다크 2
2021-09-01
11412
610
20210904
-------------------------
코다
2021-08-31
6244
412
20210904
-------------------------
퍼피 구조대 더 무비
2021-08-18
2159
155
20210904
-------------------------
귀문
2021-08-25
1816
183
20210904
-------------------------
여름날 우리
2021-08-25
1808
132
20210904
-------------------------
프리 가이 
2021-08-11
1316
70
20210904
-------------------------
샹치와 텐 링즈의 전설
2021-09-01
208990
6992
20210905
-------------------------
인질
2021-08-18
52562
2669
20210905
-------------------------
모가디슈
20

In [166]:
# 날짜형으로 바꾸고 싶은 컬럼을 pd.to_datetime()에 넣어주면 변환됩니다.
merged_data['openDt'] = merged_data['openDt'].astype('datetime64[ns]')

ParserError: String does not contain a date:  

In [43]:
# 날짜가 안 들어가는 이유는, 개봉날짜가 빈 칸으로(날짜형식이 아닌) 집계되는 부분이 있어서입니다.
# 따라서 결측치나 이상값을 집계가능한 값으로 돌려놓고(1900-01-01) 다시 insert를 하면 됩니다.
# 반복문으로 인덱스번호, 날짜 형식으로 출력하도록 해서 몇 번 인덱스가 결측치인지 파악해보겠습니다.
for i in range(len(merged_data)):
    print(i, merged_data['openDt'].iloc[i])

0 2021-07-28 00:00:00
1 2021-08-04 00:00:00
2 2021-07-21 00:00:00
3 2021-07-07 00:00:00
4 2021-08-05 00:00:00
5 2021-07-28 00:00:00
6 2021-07-28 00:00:00
7 2021-08-04 00:00:00
8 2021-08-05 00:00:00
9 2021-07-14 00:00:00
10 2021-07-28 00:00:00
11 2021-08-04 00:00:00
12 2021-07-21 00:00:00
13 2021-07-07 00:00:00
14 2021-07-28 00:00:00
15 2021-08-11 00:00:00
16 2021-08-05 00:00:00
17 2021-07-28 00:00:00
18 2021-08-05 00:00:00
19 2021-07-14 00:00:00
20 2021-07-28 00:00:00
21 2021-08-04 00:00:00
22 2021-07-21 00:00:00
23 2021-07-07 00:00:00
24 2021-08-05 00:00:00
25 2021-07-28 00:00:00
26 2021-08-11 00:00:00
27 2021-07-28 00:00:00
28 2021-08-05 00:00:00
29 2021-07-14 00:00:00
30 2021-08-11 00:00:00
31 2021-07-28 00:00:00
32 2021-08-11 00:00:00
33 2021-08-04 00:00:00
34 2021-07-21 00:00:00
35 2021-07-07 00:00:00
36 2021-08-05 00:00:00
37 2021-08-05 00:00:00
38 2021-08-04 00:00:00
39 2021-08-19 00:00:00
40 2021-08-11 00:00:00
41 2021-07-28 00:00:00
42 2021-08-11 00:00:00
43 2021-08-04 00:00:0

In [44]:
# 조건색인으로 날짜가 빈 요소만 체크해서 1900-01-01로 집어넣어보세요
merged_data['openDt'][merged_data['openDt'] == ' '] = '1900-01-01'

C:\Users\ict03-00\AppData\Local\Temp\ipykernel_23288\2656411603.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data['openDt'][merged_data['openDt'] == ' '] = '1900-01-01'


In [56]:
merged_data['openDt'].contain("'")

AttributeError: 'Series' object has no attribute 'contain'

In [168]:
# 인덱스가 365일치를 겹쳐서 받아오기 때문에 중복인덱스가 생기므로, 초기화
merged_data = merged_data.reset_index()

In [169]:
merged_data

,level_0,index,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,...,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt,date
0,0,0,1,1,0,OLD,20204117,모가디슈,2021-07-28,2000829380,...,38859180,2,16579074530,196672,4041,2.1,1712375,1639,6757,20210808
1,1,1,2,2,0,OLD,20217845,더 수어사이드 스쿼드,2021-08-04,676797240,...,-127499190,-15.9,2942253950,63095,-12197,-16.2,281377,1033,3306,20210808
2,2,2,3,3,0,OLD,20218391,보스 베이비 2,2021-07-21,284206020,...,1506480,0.5,7511398580,30046,108,0.4,836805,777,1654,20210808
3,3,3,4,4,0,OLD,20191951,블랙 위도우,2021-07-07,194489780,...,-28197330,-12.7,29457331370,18740,-2673,-12.5,2908679,540,924,20210808
4,4,4,5,5,0,OLD,20218875,극장판 도라에몽: 진구의 신공룡,2021-08-05,88917620,...,-4415810,-4.7,337195080,9837,-612,-5.9,38669,502,781,20210808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3645,3645,5,6,6,1,OLD,20209654,헤어질 결심,2022-06-29,169174033,...,-32677140,-16.2,18385204948,15203,-2798,-15.5,1775050,266,426,20220807
3646,3646,6,7,7,-1,OLD,20208446,외계+인 1부,2022-07-20,123965118,...,-98324419,-44.2,15629862451,11690,-7177,-38,1496935,374,499,20220807
3647,3647,7,8,8,0,OLD,20226107,극장판 도라에몽: 진구의 우주소전쟁 리틀스타워즈 2021,2022-08-03,77727199,...,2170399,2.9,346253298,7903,248,3.2,36417,357,466,20220807
3648,3648,8,9,9,0,OLD,20225237,명탐정 코난: 할로윈의 신부,2022-07-13,44635198,...,623298,1.4,4598589872,4122,13,0.3,442570,91,115,20220807


In [121]:
# 특정 요소에 특정한 문자가 포함되었는지는
# .str.conatins("조회문자") 형식으로 확인 가능합니다.
merged_data['movieNm'][merged_data['movieNm'].str.contains("'")]

2286    뱅드림! 팝핀' 드림!
2297    뱅드림! 팝핀' 드림!
2308    뱅드림! 팝핀' 드림!
2317    뱅드림! 팝핀' 드림!
2379    뱅드림! 팝핀' 드림!
Name: movieNm, dtype: object

In [170]:
# 위의 방식으로 우리가 수정해야할 movieNm의 인덱스번호만 얻어올 수 있음
target_rnum = merged_data[['rnum','movieNm']][merged_data['movieNm'].str.contains("'")].index

In [171]:
target_rnum

Int64Index([2286, 2297, 2308, 2317, 2379], dtype='int64')

In [135]:
for i in target_rnum:
   merged_data.iloc[i]['movieNm'] = merged_data['movieNm'][merged_data['movieNm'].str.contains("'")].str.replace("'"," ")

C:\Users\ict03-00\AppData\Local\Temp\ipykernel_23288\4254321146.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data.iloc[i]['movieNm'] = merged_data['movieNm'][merged_data['movieNm'].str.contains("'")].str.replace("'"," ")


In [172]:
merged_data.iloc[target_rnum]['movieNm']

2286    뱅드림! 팝핀' 드림!
2297    뱅드림! 팝핀' 드림!
2308    뱅드림! 팝핀' 드림!
2317    뱅드림! 팝핀' 드림!
2379    뱅드림! 팝핀' 드림!
Name: movieNm, dtype: object

In [ ]:
# apply()를 이용한 전처리
# .apply()는 데이터프레임에 적용해서, 사용자가 전처리 형식을 직접 지정해 적용할 수 있도록 도와줍니다.
# 함수를 이용해 처리합니다.
# 지금같은경우, '가 포함된 문자열을 '를 제거한 형태로 바꾸려 하므로 해당 로직을 함수로 만들어 저장하고
# 그 다음 적용하고 싶은 컬럼에 .apply()를 걸어 적용하면됩니다.

In [173]:
# 요소를 읽어서, '가 포함된 경우 삭제한 다음 리턴하는 함수 작성
def deletion_char(target):
    return target.replace("'", "")

In [174]:
# 함수가 만들어졌으면, 수정해줄 컬럼을 지정하고, 거기에 apply()를 쓴 다음 함수명을 내부에 파라미터로 줍니다.
merged_data['movieNm'] = merged_data['movieNm'].apply(deletion_char)

In [175]:
merged_data.iloc[target_rnum]

,level_0,index,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,...,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt,date
2286,2286,6,7,7,0,NEW,20224069,뱅드림! 팝핀 드림!,2022-03-24,13856540,...,13856540,100,13856540,1444,1444,100,1444,104,210,20220324
2297,2297,7,8,8,-1,OLD,20224069,뱅드림! 팝핀 드림!,2022-03-24,8210700,...,-5645840,-40.7,22067240,774,-670,-46.4,2218,89,165,20220325
2308,2308,8,9,9,-1,OLD,20224069,뱅드림! 팝핀 드림!,2022-03-24,14980130,...,6769430,82.4,37047370,1531,757,97.8,3749,105,197,20220326
2317,2317,7,8,8,1,OLD,20224069,뱅드림! 팝핀 드림!,2022-03-24,12110600,...,-2869530,-19.2,49157970,1216,-315,-20.6,4965,102,184,20220327
2379,2379,9,10,10,5,OLD,20224069,뱅드림! 팝핀 드림!,2022-03-24,8868760,...,5754260,184.8,68896440,856,563,192.2,6916,61,78,20220402


In [176]:
# apply로 또 해결합니다
def set_date(target):
    if(target == ' '):
        return '1900-01-01'
    return target

In [177]:
# 조건색인으로 2021년 10월 23일 데이터를 다시 봅니다.
merged_data['openDt'] = merged_data['openDt'].apply(set_date)

In [178]:
merged_data['openDt']

0       2021-07-28
1       2021-08-04
2       2021-07-21
3       2021-07-07
4       2021-08-05
           ...    
3645    2022-06-29
3646    2022-07-20
3647    2022-08-03
3648    2022-07-13
3649    2022-08-10
Name: openDt, Length: 3650, dtype: object

In [179]:
#위의 반복문 코드를 응용해 전체 데이터를 다 DB에 넣고 확인해주세요.
for i in range(len(merged_data)):
    selected_row = merged_data.iloc[i]
    cur.execute("INSERT INTO movie_tbl (movieNm, openDt, audiCnt, showCnt, dataDate) VALUES ('%s', '%s', %s, %s, '%s')" % (selected_row['movieNm'], 
                                                                                                                           selected_row['openDt'], 
                                                                                                                           selected_row['audiCnt'], 
                                                                                                                           selected_row['showCnt'], 
                                                                                                                           selected_row['date']))
con.commit()

In [ ]:
con.commit()

In [180]:
#SELECT구문 날려서 DataFrame으로 다시받아오기
cur.execute("SELECT * FROM movie_tbl")
movie_tbl = cur.fetchall()
pd.DataFrame(movie_tbl)

,rnum,movieNm,openDt,audiCnt,showCnt,dataDate
0,7683,모가디슈,2021-07-28,196672,6757,2021-08-08
1,7684,더 수어사이드 스쿼드,2021-08-04,63095,3306,2021-08-08
2,7685,보스 베이비 2,2021-07-21,30046,1654,2021-08-08
3,7686,블랙 위도우,2021-07-07,18740,924,2021-08-08
4,7687,극장판 도라에몽: 진구의 신공룡,2021-08-05,9837,781,2021-08-08
...,...,...,...,...,...,...
3645,11328,헤어질 결심,2022-06-29,15203,426,2022-08-07
3646,11329,외계+인 1부,2022-07-20,11690,499,2022-08-07
3647,11330,극장판 도라에몽: 진구의 우주소전쟁 리틀스타워즈 2021,2022-08-03,7903,466,2022-08-07
3648,11331,명탐정 코난: 할로윈의 신부,2022-07-13,4122,115,2022-08-07


# 위 로직을 똑같이 오라클에서도 활용할 수 있습니다. 직접 해 보세요.

In [182]:
import cx_Oracle

In [187]:
# Oracle 연동
# 11버전은 localhost:1521/XE
# 18버전은 localhost:1521/XEPDB1
con_oracle = cx_Oracle.connect("mytest", "mytest", "localhost:1521/XE")

In [185]:
# 커서 얻어온 뒤부터는 mysql과 동일하게 cur.fetchall() 등을 이용해 처리할 수 있음.
cur_oracle = con_oracle.cursor()

In [186]:
# oracle을 켜서 테이블을 똑같이 만들어주시고, 위의 mysql로직에서 얻어온 merged_data를 역시
# oracle DB에도 적재해주시고 다시 꺼내서 DataFrame화를 시켜주시면 됩니다.
#위의 반복문 코드를 응용해 전체 데이터를 다 DB에 넣고 확인해주세요.
for i in range(len(merged_data)):
    selected_row = merged_data.iloc[i]
    cur_oracle.execute("INSERT INTO movie_tbl VALUES (movieNum.nextval, '%s', '%s', %s, %s, '%s')" % (selected_row['movieNm'], 
                                                                                                                           selected_row['openDt'], 
                                                                                                                           selected_row['audiCnt'], 
                                                                                                                           selected_row['showCnt'], 
                                                                                                                           selected_row['date']))
con_oracle.commit()

In [189]:
#SELECT구문 날려서 DataFrame으로 다시받아오기
cur_oracle.execute("SELECT * FROM movie_tbl")
movie_tbl = cur_oracle.fetchall()
movie_db_data = pd.DataFrame(movie_tbl)

In [191]:
movie_db_data.columns = ['rnum', 'movieNm', 'openDt', 'audiCnt', 'showcnt', 'dataDate']

In [192]:
movie_db_data

,rnum,movieNm,openDt,audiCnt,showcnt,dataDate
0,1,모가디슈,2021-07-28,196672,6757,2021-08-08
1,2,더 수어사이드 스쿼드,2021-08-04,63095,3306,2021-08-08
2,3,보스 베이비 2,2021-07-21,30046,1654,2021-08-08
3,4,블랙 위도우,2021-07-07,18740,924,2021-08-08
4,5,극장판 도라에몽: 진구의 신공룡,2021-08-05,9837,781,2021-08-08
...,...,...,...,...,...,...
3645,3646,헤어질 결심,2022-06-29,15203,426,2022-08-07
3646,3647,외계+인 1부,2022-07-20,11690,499,2022-08-07
3647,3648,극장판 도라에몽: 진구의 우주소전쟁 리틀스타워즈 2021,2022-08-03,7903,466,2022-08-07
3648,3649,명탐정 코난: 할로윈의 신부,2022-07-13,4122,115,2022-08-07
